In [1]:
# !pip3 install altair

In [2]:
import pandas as pd
import altair as alt
alt.renderers.enable('notebook')

import numpy as np
import gzip
import json
import pickle

In [3]:
with gzip.open("./Data/study/study.json.gz", 'rt') as zipfile:
    studyProvenanceDB = json.load(zipfile)
    
with gzip.open("./Data/task/task.json.gz", 'rt') as zipfile:
    taskProvenanceDB = json.load(zipfile)

In [4]:
with open("Data/codeDict.pickle", "rb") as f:
  codeDict = pickle.load(f)

In [5]:
def getFromMainDB(path):
  return studyProvenanceDB[path]
  
def getFromGraphDB(path):
  res = None
  for p in path.split("/"):
    if res is None:
      res = taskProvenanceDB[p]
    else:
      res = res[p]
  return res


In [6]:
finalFeedbackQuestions = [
    {

    "question": "How helpful is the “individual selection” of points feature?",
    "lowText": "Not Helpful",
    "highText": "Very Helpful"
  },
  {

    "question": "How helpful is the rectangle selection feature?",
    "lowText": "Not Helpful",
    "highText": "Very Helpful"
  },
  {

    "question": "How helpful is the paintbrush feature?",
    "lowText": "Not Helpful",
    "highText": "Very Helpful"
  },
  {

    "question": "How easy was the prediction interface to use?",
    "lowText": "Very Difficult",
    "highText": "Very Easy"
  },
  {

    "question": "How accurate did you find the predictions?",
    "lowText": "Not Accurate",
    "highText": "Very Accurate"
  },
  {

    "question": "How helpful did you find the predictions?",
    "lowText": "Not Helpful",
    "highText": "Very Helpful"
  },
  {

    "question": "Do you prefer user-driven or computer supported selections?",
    "lowText": "Strongly prefer user-driven",
    "highText": "Strongly prefer computer supported"
  },
  {
    "question": "How experienced are you with scatterplots?",
    "lowText": "Not experienced",
    "highText": "Very experienced"
  },
  {
    "question": "How experienced are you with statistics?"
   ,
    "lowText": "Not experienced",
    "highText": "Very experienced"
  }
]

In [7]:
outlierBaseTypeDict = {
    "out_easy_training_1":"cluster",
"out_easy_task_1":"cluster",
"out_easy_task_2":"cluster",
"out_easy_task_3":"linear",
"out_easy_task_4":"linear",
"out_easy_task_5":"curve",
"out_med_training_1":"cluster",
"out_med_task_1":"cluster",
"out_med_task_2":"cluster",
"out_med_task_3":"linear",
"out_med_task_4":"linear",
"out_med_task_5":"curve",
"out_hard_training_1":"cluster",
"out_hard_task_1":"cluster",
"out_hard_task_2":"cluster",
"out_hard_task_3":"linear",
"out_hard_task_4":"linear",
"out_hard_task_5":"curve"
}

In [8]:
from datetime import datetime
def parseCode(data, stop=True):
    root = ""
    nodes = data['nodes']
    for i,v in nodes.items():
        if 'parent' in v:
            nodes[v['parent']]['children'] = []
            nodes[v['parent']]['children'].append(v['id'])
        if v['label'] == "Root":
            root = v['id']
    temp = root
    taskDict = {}
    tasks = []

    while True:
        state = nodes[temp]['state']
        children = []
        if 'children' in nodes[temp]:
            children = nodes[temp]['children']
        taskId = state['taskId']
        if taskId not in taskDict:
            taskDict[taskId] = []
        taskDict[taskId].append(nodes[temp])
#         if "Final Feedback" in nodes[temp]['label']:
#             pprint.pprint(nodes[temp]['label'])
#             pprint.pprint(nodes[temp]['state'])
        if len(children) == 0: break
        temp = children[0]


    for taskId, taskProvenance in taskDict.items():
        taskDets = {}
        if taskId == "-1":
            continue
        taskProvenance = sorted(taskProvenance, key=lambda x: x['metadata']['createdOn'])
        taskDets['id'] = taskId
        startNode = next(filter(lambda x: "Start task:" in x['label'], taskProvenance), False)
        endNode = next(filter(lambda x: "Complete task:" in x['label'], taskProvenance), False)
        if(type(endNode) == bool):
            continue

        taskStartTime  = startNode['metadata']['createdOn']
        taskEndTime = endNode['metadata']['createdOn']
        timeRequired = datetime.fromtimestamp(taskEndTime / 1000) - datetime.fromtimestamp(taskStartTime / 1000)

        state = endNode['state']

        task = state['task']

        autoCompleteUsed = False
        rankOfPredictionUsed = -1
        totalPredictionsMade = -1
        selectionDetails = {
            'rectangular': {
                'timestamps': [],
                'occurences': 0
            },
            'paint': {
                'timestamps': [],
                'occurences': 0
            },
            'cleared': {
                'timestamps': [],
                'occurences': 0
            },
            'invert': {
                'timestamps': [],
                'occurences': 0
            }
        }
        selectedPrediction = ""

        graphPath = state['graph']
        graph = {}
        if task['manual'] == "supported" and stop:
          graph = getFromGraphDB(graphPath)
          gNodes = graph['nodes']
          for i,v in gNodes.items():
            label = v['label']
            time = v['metadata']['createdOn']
            if 'Point Selection' in label:
              selectionDetails['paint']['occurences'] += 1
              selectionDetails['paint']['timestamps'].append(time)
            if 'Add brush to plot' in label:
              selectionDetails['rectangular']['occurences'] += 1
              selectionDetails['rectangular']['timestamps'].append(time)
            if 'Clear all' in label:
              selectionDetails['cleared']['occurences'] += 1
              selectionDetails['cleared']['timestamps'].append(time)
            if 'Invert' in label:
              selectionDetails['invert']['occurences'] += 1
              selectionDetails['invert']['timestamps'].append(time)
            if 'turnedPrediction' in v['state'].keys():
              selectedPrediction = v['state']['turnedPrediction']
              if 'extra' in gNodes[v['parent']]['artifacts']:
                extra = gNodes[v['parent']]['artifacts']['extra']
                extra = list(extra.values())
                if len(extra) > 0 or '0' in extra:
                  if 'predictions' in extra[0]['e']['predictionSet']:
                    predictions = extra[0]['e']['predictionSet']['predictions']
                    predictions = list(predictions.values())
                    predictions = sorted(predictions, key=lambda x: x['rank'], reverse=True)
                    selectedIdxes = [i for i,x in enumerate(predictions) if x['intent'] == selectedPrediction]
                    if len(selectedIdxes) > 0:
                      autoCompleteUsed = True
                      rankOfPredictionUsed = selectedIdxes[0] + 1
                      totalPredictionsMade = len(predictions)
                      nameArr =  selectedPrediction.split(":")
                      intentName = nameArr[2]
                      if "Regression" in intentName:
                        intentName = f"{intentName} - {nameArr[3]}"
                      selectedPrediction = intentName
                      
    
        interactionDetails = {}
        interactionDetails['autoCompleteUsed'] = autoCompleteUsed
        interactionDetails['rankOfPredictionUsed'] = rankOfPredictionUsed
        interactionDetails['totalPredictionsMade'] = totalPredictionsMade
        interactionDetails['selectionDetails'] = selectionDetails
        interactionDetails['selectedPrediction'] = selectedPrediction

        taskDets['type'] = task['type']
        taskDets['training'] = task['training']
        taskDets['task'] = task['task']
        taskDets['dataset'] = task['dataset']
        taskDets['difficulty'] = task['difficulty']
        taskDets['user-driven'] = task['manual']
        taskDets['user_confidence'] = state['confidenceScore']
        taskDets['user_difficulty'] = state['difficultyScore']
        taskDets['user_task_feedback'] = state['feedback']
        taskDets['selections'] = state['selections']
        taskDets['graph'] = endNode['state']['graph']
        taskDets['interactionDetails'] = interactionDetails

        taskDets['startedAt'] = str(datetime.fromtimestamp(taskStartTime/1000))
        taskDets['completedAt'] = str(datetime.fromtimestamp(taskEndTime/1000))
        taskDets['timeRequired'] = str(timeRequired)
        tasks.append(taskDets)
    # break

    return pd.DataFrame(tasks)

# res = getStudyResultsFor(prov_runs[6])
# pprint.pprint(res['data']['tasks']["33"])

In [9]:
def getStudyProvenanceFor(data):
    nodes = data['nodes']

    newNode = []

    for n in nodes.values():
        label = n['label']
        time = n['metadata']['createdOn']
        state = n['state']
        state['label'] = label
        state['time'] = time
        newNode.append(state)

    newNode = sorted(newNode, key=lambda x: x['time'])
    return newNode[0]['participantId'], newNode

def getStudyProvenanceForMultiple(dataArr):
    events = []
    for d in dataArr:
        event = {}
        id,evt = getStudyProvenanceFor(d)
        event['id'] = id
        event['data'] = evt
        events.append(event)
    return events

In [10]:
from scipy.spatial.distance import jaccard
from numpy import interp
import copy

def parseFeedback(scores):
    feed = []
    for idx,score in enumerate(scores):
        question = finalFeedbackQuestions[idx]
        question['score'] = int(score) + 1
        feed.append(question)
    return feed
        

def getFinalFeedback(data):
    for i,v in data['nodes'].items():
        if "Final Feedback" == v['label']:
            return {
                 'scores': parseFeedback(v['state']['finalFeedbackArr']), 'comment': v['state']["finalFeedbackComment"]
            }
    return []

def getAccuracy2(user, code):
    userArr = [1 if i in user else 0 for i in range(len(code))]
    userCorrect = 0
    userPenalize = 0
    maxPossiblePoints = 0
    minPossiblePoints = 0
    
    for u,c in zip(userArr, code):
        if c >= 4:
            maxPossiblePoints += 1
        if c == 0:
            minPossiblePoints -= 1
        if c >= 4 and u == 1:
            userCorrect += 1
        if c == 0 and u == 1:
            userPenalize -= 1
    
    accuracy = interp(userCorrect + userPenalize, [minPossiblePoints, maxPossiblePoints], [0,1])
    
    return round(accuracy, 2)

def getAccuracy(user, code, id):
  userArr = [1 if i in user else 0 for i in range(len(code))]

  ua = []
  ca = []

  for u,c in zip(userArr, code):
    if c >= 4 or c <= 1:
      ua.append(u)
      ca.append(c)
  ca = [1 if i > 1 else 0 for i in ca]

  return 1 - jaccard(ua,ca)

def getStudyResultsFor(data, idx = 0):
    df = parseCode(data)
    finalFeedback = {}

    feedbackScores = getFinalFeedback(data)

    partId = ""
    for i in data['nodes'].keys():
        partId = data['nodes'][i]['state']['participantId']
        break
    if len(partId) == 0: return

    finalData = {}
    finalData['finalFeedback'] = copy.deepcopy(feedbackScores)

    demographics = {}
    info = pmd[pmd['participant_id'] == partId]
    # info = prolificMetaData[prolificMetaData['participant_id'] == partId]
    if info.shape[0] > 0:
        demographics['country_birth'] = info["Country of Birth"].values[0]
        demographics['country_residence'] = info["Current Country of Residence"].values[0]
        demographics['employment'] = info["Employment Status"].values[0]
        demographics['nationality'] = info["Nationality"].values[0]
        demographics['sex'] = info["Sex"].values[0]
        demographics['student_status'] = info["Student Status"].values[0]
        finalData['demographics'] = demographics
    
    overAllAcc = 0

    
    finalData['tasks'] = {}

    for i,row in df.iterrows():
        id = row['id']
        taskDets = {**row.to_dict()}
        selections = row['selections']
        # del taskDets['selections']
        rel_code = []
        try:
          rel_code = codeDict[int(id)]
        except:
          rel_code = [0] * len(selections)
        accuracy = getAccuracy(selections, rel_code, id)
        taskDets['accuracy'] = accuracy
        overAllAcc += accuracy
        taskDets["group"] = f"{taskDets['difficulty']}_{taskDets['user-driven']}_{taskDets['type']}"
        if taskDets['type'] == 'outlier':
          taskDets['group'] = f"{taskDets['group']}_{outlierBaseTypeDict[taskDets['dataset']]}"
        finalData['tasks'][id] = taskDets

    finalData['participantId'] = partId
    finalData['avgAcc'] = overAllAcc / df.shape[0]
    return {'data': finalData}

def getStudyResults(arr):
    data = []
    for i,d in enumerate(arr):
        data.append(getStudyResultsFor(d,i))
    return data

In [11]:
def getShape(arr):
  prov_runs = []
  count = 1
  for part in arr:
    count += 1
    data = getFromMainDB(part)
    studyId = list(data.keys())[0]
    sessionId = list(data[studyId].keys())[-1]
    d = data[studyId][sessionId]
    prov_runs.append(d)
  return prov_runs


In [12]:
partInfo = pd.read_csv("Data/partInfo.csv")
pmd = partInfo
partIds = partInfo.loc[:, ['participant_id']].values[:,0]

prov_runs = getShape(partIds)

In [13]:
studyProvenance = getStudyProvenanceForMultiple(prov_runs)
len(studyProvenance)

128

In [15]:
processed_results = getStudyResults(prov_runs)
len(processed_results)

128